In [43]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import os

In [39]:
!unzip -q /content/turbo-codes-8psk/psk8turbo_snr.zip

In [50]:
class_names = ["CONV", "TPC", "BCH", "TURBO"]
x = []
y = []
for i in os.listdir():
  if "csv" in i and "conv" in i:
    print("Compiling file :", i)
    conv_0 = pd.read_csv(i, header=None)
    for j in range(100):
      x.append(conv_0.iloc[:200, j].values)
      y.append(0)

Compiling file : conv_5snr.csv
Compiling file : conv_4snr.csv
Compiling file : conv_6snr.csv
Compiling file : conv_0snr.csv
Compiling file : conv_3snr.csv
Compiling file : conv_1snr.csv
Compiling file : conv_10snr.csv
Compiling file : conv_2snr.csv
Compiling file : conv_8snr.csv
Compiling file : conv_7snr.csv


In [51]:
for i in os.listdir("/content/turbo"):
  if "csv" in i:
    print("Compiling file :", i)
    turbo_0 = pd.read_csv("/content/turbo/"+i, header=None)
    for j in range(100):
      x.append(turbo_0.iloc[:200, j].values)
      y.append(1)

Compiling file : tpc_5snr.csv
Compiling file : tpc_3snr.csv
Compiling file : tpc_8snr.csv
Compiling file : tpc_10snr.csv
Compiling file : tpc_9snr.csv
Compiling file : tpc_6snr.csv
Compiling file : tpc_0snr.csv
Compiling file : tpc_2snr.csv
Compiling file : tpc_1snr.csv
Compiling file : tpc_7snr.csv
Compiling file : tpc_4snr.csv


In [52]:
for i in os.listdir("/content/bch"):
  if "csv" in i:
    print("Compiling file :", i)
    turbo_0 = pd.read_csv("/content/bch/"+i, header=None)
    for j in range(100):
      x.append(turbo_0.iloc[:200, j].values)
      y.append(2)

Compiling file : bch8psk_6snr.csv
Compiling file : bch8psk_5snr.csv
Compiling file : bch8psk_2snr.csv
Compiling file : bch8psk_1snr.csv
Compiling file : bch8psk_3snr.csv
Compiling file : bch8psk_0snr.csv
Compiling file : bch8psk_10snr.csv
Compiling file : bch8psk_7snr.csv
Compiling file : bch8psk_9snr.csv
Compiling file : bch8psk_8snr.csv
Compiling file : bch8psk_4snr.csv


In [ ]:
for i in os.listdir():
  if "psk8turbo" in i:
    print("Compiling file :", i)
    conv_0 = pd.read_csv(i, header=None)
    print(conv_0.head())
    for j in range(100):
      x.append(conv_0.iloc[:200, j].values)
      y.append(3)
x = np.array(x)
x.shape

In [22]:
x_data = []
for i in x:
  x_data.append([[x] for x in i])

x_data = np.array(x_data)

In [23]:
x_data.shape

(3200, 200, 1)

In [25]:
y_data = np.array(y)

In [26]:
inp_layer = keras.layers.Input(shape=(200,1))

#conv1-64, 3x1
x = keras.layers.Conv1D(filters=64, kernel_size=3)(inp_layer)
x_act = keras.layers.Activation(tf.nn.relu)(x)

#conv1-64, 3x1
x = keras.layers.Conv1D(filters=64, kernel_size=3)(x_act)
x_act = keras.layers.Activation(tf.nn.relu)(x)

#conv1-128, 3x1
x = keras.layers.Conv1D(filters=128, kernel_size=3)(x_act)
x_act = keras.layers.Activation(tf.nn.relu)(x)

#global average pooling
gap = keras.layers.GlobalAveragePooling1D()(x_act)

#dense
d1 = keras.layers.Dense(64, activation="relu")(gap)
d2 = keras.layers.Dense(64, activation="relu")(d1)
out_layer = keras.layers.Dense(3, activation="sigmoid")(d2)

model_test1 = keras.Model(inp_layer, out_layer)

model_test1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200, 1)]          0         
                                                                 
 conv1d (Conv1D)             (None, 198, 64)           256       
                                                                 
 activation (Activation)     (None, 198, 64)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 196, 64)           12352     
                                                                 
 activation_1 (Activation)   (None, 196, 64)           0         
                                                                 
 conv1d_2 (Conv1D)           (None, 194, 128)          24704     
                                                                 
 activation_2 (Activation)   (None, 194, 128)          0     

In [27]:
model_test1.compile(
    optimizer="adam",
    loss = "sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [28]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.25, shuffle=True, stratify=y_data)
x_t, x_v, y_t, y_v = train_test_split(x_test, y_test, test_size=0.5, shuffle=True, stratify=y_test)

x_train.shape, x_t.shape, x_v.shape, y_train.shape, y_t.shape, y_v.shape

((2400, 200, 1), (400, 200, 1), (400, 200, 1), (2400,), (400,), (400,))

In [31]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint("model_8psk.h5"),
    tf.keras.callbacks.TensorBoard("logs_8psk_training")
]

In [32]:
model_test1.fit(x_train, y_train, epochs=100, validation_data=(x_v, y_v), callbacks=callbacks)

Epoch 1/100
75/75 [==============================] - 4s 50ms/step - loss: 0.4574 - accuracy: 0.6954 - val_loss: 0.4536 - val_accuracy: 0.6950
Epoch 2/100
 3/75 [>.............................] - ETA: 2s - loss: 0.4682 - accuracy: 0.6771

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


75/75 [==============================] - 4s 50ms/step - loss: 0.4514 - accuracy: 0.6988 - val_loss: 0.4457 - val_accuracy: 0.6900
Epoch 3/100
75/75 [==============================] - 3s 43ms/step - loss: 0.4553 - accuracy: 0.6817 - val_loss: 0.4469 - val_accuracy: 0.7250
Epoch 4/100
75/75 [==============================] - 3s 37ms/step - loss: 0.4514 - accuracy: 0.6779 - val_loss: 0.4456 - val_accuracy: 0.7175
Epoch 5/100
75/75 [==============================] - 3s 38ms/step - loss: 0.4510 - accuracy: 0.7063 - val_loss: 0.4542 - val_accuracy: 0.7125
Epoch 6/100
75/75 [==============================] - 3s 42ms/step - loss: 0.4522 - accuracy: 0.6896 - val_loss: 0.4488 - val_accuracy: 0.7175
Epoch 7/100
75/75 [==============================] - 4s 50ms/step - loss: 0.4511 - accuracy: 0.6983 - val_loss: 0.4442 - val_accuracy: 0.7175
Epoch 8/100
75/75 [==============================] - 3s 36ms/step - loss: 0.4484 - accuracy: 0.7021 - val_loss: 0.4558 - val_accuracy: 0.6875
Epoch 9/100
75/75 

In [34]:
model_test1.evaluate(x_t, y_t)

13/13 [==============================] - 0s 26ms/step - loss: 0.3331 - accuracy: 0.8450


[0.33308303356170654, 0.8450000286102295]

In [35]:
model_test1.save("model_8psk_90%acc")

In [36]:
!zip -r model_8psk.zip /content/model_8psk_90%acc

  adding: content/model_8psk_90%acc/ (stored 0%)
  adding: content/model_8psk_90%acc/variables/ (stored 0%)
  adding: content/model_8psk_90%acc/variables/variables.index (deflated 65%)
  adding: content/model_8psk_90%acc/variables/variables.data-00000-of-00001 (deflated 10%)
  adding: content/model_8psk_90%acc/keras_metadata.pb (deflated 91%)
  adding: content/model_8psk_90%acc/saved_model.pb (deflated 88%)
  adding: content/model_8psk_90%acc/assets/ (stored 0%)
  adding: content/model_8psk_90%acc/fingerprint.pb (stored 0%)


In [37]:
!zip -r logs_8psk.zip /content/logs_8psk_training

  adding: content/logs_8psk_training/ (stored 0%)
  adding: content/logs_8psk_training/validation/ (stored 0%)
  adding: content/logs_8psk_training/validation/events.out.tfevents.1703029798.97196dc77dca.1537.1.v2 (deflated 79%)
  adding: content/logs_8psk_training/train/ (stored 0%)
  adding: content/logs_8psk_training/train/events.out.tfevents.1703029794.97196dc77dca.1537.0.v2 (deflated 85%)
